### Input data

1. The predicted probabilities file `best_embs_full.py` can be obtained after running `train.py`.
2. `processed_data_test.csv` is from `prepare_data.ipynb`.

In [ ]:
test_embs = np.load('best_embs_full.npy')
test_interaction_df = pd.read_csv('processed_data_test.csv', index_col=0)
test_interaction_df['prob'] = test_embs

In [ ]:
test_interaction_df.to_csv('covid_results.csv')

In [ ]:
targets = test_interaction_df['target'].tolist()
labels = test_interaction_df['groups'].tolist()
names = test_interaction_df['names'].tolist()
origins = test_interaction_df['origins'].tolist()

In [ ]:
from sklearn.metrics import precision_recall_curve, auc

def calculate_auprc_per_label(y_true, y_pred, label_types):
    # Create a dictionary to store true labels and predictions for each label type
    label_data = {}
    
    for label, true_val, pred_val in zip(label_types, y_true, y_pred):
        if label not in label_data:
            label_data[label] = {'y_true': [], 'y_pred': []}
        
        label_data[label]['y_true'].append(true_val)
        label_data[label]['y_pred'].append(pred_val)
    
    # Calculate AUPRC for each label type
    auprc_per_label = {}
    
    for label, data in label_data.items():
        if len(data['y_true']) > 3:
            precision, recall, _ = precision_recall_curve(data['y_true'], data['y_pred'])
            auprc = auc(recall, precision)
            auprc_per_label[label] = (auprc, np.unique(data['y_true'], return_counts=True)[1])
    return auprc_per_label

In [ ]:
calculate_auprc_per_label(targets, test_embs, labels)

In [ ]:
calculate_auprc_per_label(targets, test_embs, origins)